In [42]:
!pip install elsapy

In [31]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
from elsapy.elsprofile import ElsAuthor
from elsapy.elsdoc import FullDoc
from elsapy.elsdoc import AbsDoc
import json
import pandas as pd

In [32]:
# Initialize the Scopus client
client = ElsClient(api_key="737506d5b1185510c2c6a0f9f31f983f")  # Replace 'your_api_key_here' with your actual API key

In [33]:
# Define the search query
search_query = ElsSearch(query='TITLE-ABS-KEY("Covid-19")', index='scopus')

In [15]:
# Execute the search
res = search_query.execute(client)

None


In [17]:
# Function to perform a search and handle pagination
def fetch_all_results(client, query, max_results=25):
    all_results = []
    start = 0
    count = 100  # Number of results per page (Scopus API default limit)
    
    while start < max_results:
        search_query = ElsSearch(query=query, index='scopus')
        search_query.execute(client, count=count)

        if not search_query.results:
            break

        all_results.extend(search_query.results)
        start += count

        # Check if the last batch of results is less than the count, meaning we've reached the end
        if len(search_query.results) < count:
            break

    return all_results


In [34]:
# List of topics to search for
topics = [
    # 'artificial intelligence',
    # 'machine learning',
    # 'data science',
    # 'natural language processing',
    # 'computer vision',
    # 'big data',
    # 'cloud computing',
    # 'cybersecurity',
    # 'blockchain',
    # 'internet of things',
    # 'quantum computing',
    # 'robotics',
    # 'bioinformatics',
    # 'genomics',
    # 'renewable energy',
    # 'nanotechnology',
    # 'biotechnology',
    # 'autonomous vehicles',
    # 'augmented reality',
    'virtual reality'
]

In [35]:
# Initialize a list to store the publication data
publications = []

In [39]:
# Fetch results for each topic
for topic in topics:
    query = f'TITLE-ABS-KEY("{topic}") AND PUBYEAR AFT 2023'
    results = fetch_all_results(client, query)
    
    if results:
        for entry in results:
            title = entry.get('dc:title', 'No title available')
            authors = entry.get('dc:creator', 'No authors available')
            publication = entry.get('prism:publicationName', 'No publication name available')
            date = entry.get('prism:coverDate', 'No date available')
            citations = entry.get('citedby-count', 'No citations available')
            volume = entry.get('prism:volume', 'No volume available')
            issue = entry.get('prism:issueIdentifier', 'No issue available')
            pages = entry.get('prism:pageRange', 'No pages available')
            doi = entry.get('prism:doi', 'No DOI available')
            abstract = entry.get('dc:description', 'No abstract available')

            # Append the publication data to the list
            publications.append({
                'Topic': topic,
                    'Title': title,
                    'Authors': authors,
                    'Publication': publication,
                    'Date': date,
                    'Citations': citations,
                    'Volume': volume,
                    'Issue': issue,
                    'Pages': pages,
                    'DOI': doi,
                    'Abstract': abstract
            })

In [40]:
# Create a DataFrame from the publications list
df = pd.DataFrame(publications)

In [41]:
# Save the DataFrame to a CSV file

from datetime import datetime
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Define the filename with the current date and time
filename = f'data/scopus_publications_virtual reality_{current_time}.csv'

# Save the DataFrame to a CSV file with the new filename
df.to_csv(filename, index=False)

# Print the filename to confirm where the data has been saved
print(f"Results saved to {filename}")

Results saved to data/scopus_publications_virtual reality_2024-06-20_13-31-27.csv
